In [108]:
from collections import Counter
import collections, functools, operator 
import matplotlib.pyplot as plt
import math
import numpy as np

In [109]:
#read file to list of lists
#0: orthography
#1: phon w/o r dipthong char
#2: freq 1
#3: freq 2
#4: phon w/ r dipthong char
r1 = []
myFile= open( "CMUDict_Lemma_RDip_SubtlexFreqs.txt", "r" )
for aRow in myFile:
    r1.append(aRow.strip('\n').split('\t'))
myFile.close()

In [110]:
#get the number of each character in each word, multiplied by the frequency for that word
list_of_dict = []
for ii in range(0, len(r1)):
    local_dict = dict(Counter(r1[ii][4]))
    #multiply by freq
    for key in local_dict:
        local_dict[key] *= float(r1[ii][3])
    list_of_dict.append(local_dict)

In [111]:
#sum everything together to get the counts for each character in the corpus based on word frequency
result = dict(functools.reduce(operator.add, 
         map(collections.Counter, list_of_dict))) 

In [112]:
#just the monopthongy vowels
vowels = { 'i', 'I', 'e', 'E', '@', 'u', 'U', 'o', 'c', 'a', 'V', 'R' }
vowels_token = { key:value for key,value in result.items() if key in vowels}



In [113]:
#entropy of vowel system (token freq)

#first get total frequency so we can get the probability
totalfreq = sum(vowels_token.values())

#now divide the frequency of each vowel by the total frequency to get the probability
probabilities = {}
for key in vowels_token:
    probabilities[key] = vowels_token[key]/totalfreq
    
#now get the surprisal for each vowel times the probability for each vowel
surprisals = {}
for key in probabilities:
    surprisals[key] = probabilities[key]*(math.log2(1/probabilities[key]))

#now add all of those together to get the entropy
entropy_vowels_token = sum(surprisals.values())

In [114]:
entropy_vowels_token

3.3878689259664196

In [115]:
#contribution of vowels to the system (token freq)

#entropy of the entire system first
totalfreq = sum(result.values())

probabilities = {}
for key in result:
    probabilities[key] = result[key]/totalfreq
    
surprisals = {}
for key in probabilities:
    surprisals[key] = probabilities[key]*(math.log2(1/probabilities[key]))

#now add all of those together to get the entropy
entropy_total = sum(surprisals.values())

In [116]:
entropy_total

5.0571104380771805

In [117]:
#entropy of system without vowels
nonvowel = { key:value for key,value in result.items() if not key in vowels}


totalfreq = sum(nonvowel.values())

probabilities = {}
for key in nonvowel:
    probabilities[key] = nonvowel[key]/totalfreq
    
surprisals = {}
for key in probabilities:
    surprisals[key] = probabilities[key]*(math.log2(1/probabilities[key]))

#now add all of those together to get the entropy
entropy_nonvowel = sum(surprisals.values())


In [118]:
entropy_nonvowel

4.513860441153023

In [119]:
vowels_same_freq= sum(vowels_token.values())

In [121]:
onevowel = {**nonvowel, **{'V': vowels_same_freq}}

In [123]:
#entropy of system with all vowels merged
totalfreq = sum(onevowel.values())

probabilities = {}
for key in onevowel:
    probabilities[key] = onevowel[key]/totalfreq
    
surprisals = {}
for key in probabilities:
    surprisals[key] = probabilities[key]*(math.log2(1/probabilities[key]))

#now add all of those together to get the entropy
entropy_onevowel = sum(surprisals.values())


In [124]:
entropy_onevowel

4.0486482461618625

In [125]:
vowel_contr = (entropy_total - entropy_onevowel)/ entropy_total

In [126]:
vowel_contr

0.19941470613775178

## front vs. back vowels

In [127]:
#sum all frequencies of front vowels & sum all frequencies of back vowels. 
frontvowels = { 'i', 'I', 'e', 'E', '@', 'R' }
frontvowels_dict = { key:value for key,value in result.items() if key in frontvowels}
frontvowels_freq = sum(frontvowels_dict.values())

backvowels = { 'u', 'U', 'o', 'c', 'a', 'V' }
backvowels_dict = { key:value for key,value in result.items() if key in backvowels}
backvowels_freq = sum(backvowels_dict.values())


In [131]:
#make a dictionary of phonemes with all front vowels merged and all back vowels merged - choose i for front and u for back
frontback = {**nonvowel, **{'i':frontvowels_freq}, **{'u':backvowels_freq}}

In [132]:
#entropy of system with all vowels merged
totalfreq = sum(frontback.values())

probabilities = {}
for key in frontback:
    probabilities[key] = frontback[key]/totalfreq
    
surprisals = {}
for key in probabilities:
    surprisals[key] = probabilities[key]*(math.log2(1/probabilities[key]))

#now add all of those together to get the entropy
entropy_frontback = sum(surprisals.values())

In [133]:
entropy_frontback

4.340159974134184

In [134]:
vowel_contr = (entropy_total - entropy_frontback)/ entropy_total
vowel_contr

0.1417707745800379

## high vs. low vowels

In [139]:
#sum all frequencies of high vowels & sum all frequencies of low vowels. 
highvowels = { 'i', 'I', 'e', 'u', 'U', 'o' }
highvowels_dict = { key:value for key,value in result.items() if key in highvowels}
highvowels_freq = sum(highvowels_dict.values())

lowvowels = { 'E', '@', 'R', 'c', 'a', 'V' }
lowvowels_dict = { key:value for key,value in result.items() if key in lowvowels}
lowvowels_freq = sum(lowvowels_dict.values())


In [140]:
#make a dictionary of phonemes with all high vowels merged and all low vowels merged
#choose i for high and E for low
highlow = {**nonvowel, **{'i':highvowels_freq}, **{'E':lowvowels_freq}}

In [141]:
#entropy of system with all vowels merged
totalfreq = sum(highlow.values())

probabilities = {}
for key in highlow:
    probabilities[key] = highlow[key]/totalfreq
    
surprisals = {}
for key in probabilities:
    surprisals[key] = probabilities[key]*(math.log2(1/probabilities[key]))

#now add all of those together to get the entropy
entropy_highlow = sum(surprisals.values())

In [142]:
entropy_highlow

4.346237739804603

In [143]:
vowel_contr = (entropy_total - entropy_highlow)/ entropy_total
vowel_contr

0.1405689488052522

## tense vs. lax vowels

In [144]:
#sum all frequencies of tense vowels & sum all frequencies of lax vowels. 
tensevowels = { 'i', 'e', 'u', 'a', 'o', 'c' }
tensevowels_dict = { key:value for key,value in result.items() if key in tensevowels}
tensevowels_freq = sum(tensevowels_dict.values())

laxvowels = { 'I', 'E', '@', 'R', 'U', 'V' }
laxvowels_dict = { key:value for key,value in result.items() if key in laxvowels}
laxvowels_freq = sum(laxvowels_dict.values())


In [145]:
#make a dictionary of phonemes with all tense vowels merged and all lax vowels merged
#choose i for tense and I for lax
tenselax = {**nonvowel, **{'i':tensevowels_freq}, **{'I':laxvowels_freq}}

In [146]:
#entropy of system with all vowels merged
totalfreq = sum(tenselax.values())

probabilities = {}
for key in tenselax:
    probabilities[key] = tenselax[key]/totalfreq
    
surprisals = {}
for key in probabilities:
    surprisals[key] = probabilities[key]*(math.log2(1/probabilities[key]))

#now add all of those together to get the entropy
entropy_tenselax = sum(surprisals.values())

In [147]:
entropy_tenselax

4.345566678837873

In [148]:
vowel_contr = (entropy_total - entropy_tenselax)/ entropy_total
vowel_contr

0.14070164532729712

## now look at specific vowel mergers

In [150]:
# a c merger
vowelstomerge = {'a', 'c'}
vowelstomerge_dict = { key:value for key,value in result.items() if key in vowelstomerge}
vowelstomerge_freq = sum(vowelstomerge_dict.values())
nonmerged_dict = { key:value for key,value in result.items() if not key in vowelstomerge}
merged = {**nonmerged_dict, **{'<':vowelstomerge_freq}} #using < to represent merged vowels

#and to get the entropy
totalfreq = sum(merged.values())

probabilities = {}
for key in merged:
    probabilities[key] = merged[key]/totalfreq
    
surprisals = {}
for key in probabilities:
    surprisals[key] = probabilities[key]*(math.log2(1/probabilities[key]))

#now add all of those together to get the entropy
entropy_merged = sum(surprisals.values())
entropy_merged

5.0333389713024514

In [151]:
vowel_contr = (entropy_total - entropy_merged)/ entropy_total
vowel_contr

0.004700602659523387

In [152]:
# i I merger
vowelstomerge = {'i', 'I'}
vowelstomerge_dict = { key:value for key,value in result.items() if key in vowelstomerge}
vowelstomerge_freq = sum(vowelstomerge_dict.values())
nonmerged_dict = { key:value for key,value in result.items() if not key in vowelstomerge}
merged = {**nonmerged_dict, **{'<':vowelstomerge_freq}} #using < to represent merged vowels

#and to get the entropy
totalfreq = sum(merged.values())

probabilities = {}
for key in merged:
    probabilities[key] = merged[key]/totalfreq
    
surprisals = {}
for key in probabilities:
    surprisals[key] = probabilities[key]*(math.log2(1/probabilities[key]))

#now add all of those together to get the entropy
entropy_merged = sum(surprisals.values())
entropy_merged

4.980268014362102

In [153]:
vowel_contr = (entropy_total - entropy_merged)/ entropy_total
vowel_contr

0.0151949269560139

In [154]:
# u U merger
vowelstomerge = {'u', 'U'}
vowelstomerge_dict = { key:value for key,value in result.items() if key in vowelstomerge}
vowelstomerge_freq = sum(vowelstomerge_dict.values())
nonmerged_dict = { key:value for key,value in result.items() if not key in vowelstomerge}
merged = {**nonmerged_dict, **{'<':vowelstomerge_freq}} #using < to represent merged vowels

#and to get the entropy
totalfreq = sum(merged.values())

probabilities = {}
for key in merged:
    probabilities[key] = merged[key]/totalfreq
    
surprisals = {}
for key in probabilities:
    surprisals[key] = probabilities[key]*(math.log2(1/probabilities[key]))

#now add all of those together to get the entropy
entropy_merged = sum(surprisals.values())
entropy_merged

5.036590631204497

In [155]:
vowel_contr = (entropy_total - entropy_merged)/ entropy_total
vowel_contr

0.004057614941168968

In [156]:
# U V merger
vowelstomerge = {'U', 'V'}
vowelstomerge_dict = { key:value for key,value in result.items() if key in vowelstomerge}
vowelstomerge_freq = sum(vowelstomerge_dict.values())
nonmerged_dict = { key:value for key,value in result.items() if not key in vowelstomerge}
merged = {**nonmerged_dict, **{'<':vowelstomerge_freq}} #using < to represent merged vowels

#and to get the entropy
totalfreq = sum(merged.values())

probabilities = {}
for key in merged:
    probabilities[key] = merged[key]/totalfreq
    
surprisals = {}
for key in probabilities:
    surprisals[key] = probabilities[key]*(math.log2(1/probabilities[key]))

#now add all of those together to get the entropy
entropy_merged = sum(surprisals.values())
entropy_merged

5.032814304689462

In [157]:
vowel_contr = (entropy_total - entropy_merged)/ entropy_total
vowel_contr

0.004804350959943877

## WORD LEVEL

In [31]:
#all words and frequencies
word = []
for row in r1:
    word.append(row[4])
    
freq = []
for row in r1:
    freq.append(float(row[3]))


In [32]:
#there are words that are already homophones in the corpus. 
homophones = []
nonhomophones = []
for w in word:
    if word.count(w) > 1:
        homophones.append(w)
    else:
        nonhomophones.append(w)


In [33]:
#get unique homophones
homophones = set(homophones)

In [34]:
#get the indices for all the homophones
indexPosList = []
for h in homophones:
    indexPosList.append([ i for i in range(len(word)) if word[i] == h ])

In [35]:
#get the frequencies at those indices
hfreq = []
localfreq = []
for j in range(len(indexPosList)):
    for i in range(len(indexPosList[j])):
        localfreq.append(freq[indexPosList[j][i]])
    hfreq.append(np.array(localfreq))
    localfreq = []
        

In [37]:
#sum the frequencies for each homophone together
hfreqsum = []
for row in hfreq:
    hfreqsum.append(np.sum(row, 0))

In [38]:
#make a dictionary of homophones, summed frequencies 
hdict = dict(zip(homophones, hfreqsum))

In [39]:
#make a dictionary of all words that are not homophones with their frequencies
#get the indices of all the nonhomophones in the word list
indexPosList2 = []
for h in nonhomophones:
    indexPosList2.append([ i for i in range(len(word)) if word[i] == h] )


In [40]:
#get the frequencies at those indices
nhfreq = []
for row in indexPosList2:
    nhfreq.append(freq[row[0]])

In [41]:
#make a dictionary of nonhomophones and frequencies
nhdict = dict(zip(nonhomophones, nhfreq))

In [42]:
#dictionary of all words, where each unique phonemic representation appears once 
#with the frequency of all homophones summed for that representation
word_dict = {**hdict, **nhdict}

In [44]:
#entropy of system at the word level (token freq)

#first get total frequency so we can get the probability
totalfreq = sum(word_dict.values())

#now divide the frequency of each word by the total frequency to get the probability
probabilities = {}
for key in word_dict:
    probabilities[key] = word_dict[key]/totalfreq
    
#now get the surprisal for each vowel times the probability for each vowel
surprisals = {}
for key in probabilities:
    surprisals[key] = probabilities[key]*(math.log2(1/probabilities[key]))

#now add all of those together to get the entropy
entropy_words_token = sum(surprisals.values())

In [45]:
entropy_words_token

8.690740841402093

In [47]:
#now we want to see the vowel contribution at the word level.
#First, replace all vowels with V.
replace_dict = {'i':'V', 'I':'V', 'e':'V', 'E':'V', '@':'V', 'u':'V', 'U':'V', 'o':'V', 'c':'V', 'a':'V', 'R':'V'}
vowelless = []

In [48]:
# a function I found from https://codereview.stackexchange.com/questions/97318/string-replacement-using-dictionaries
def keymap_replace(
        string: str, 
        mappings: dict,
        lower_keys=False,
        lower_values=False,
        lower_string=False,
    ) -> str:
    """Replace parts of a string based on a dictionary.

    This function takes a string a dictionary of
    replacement mappings. For example, if I supplied
    the string "Hello world.", and the mappings 
    {"H": "J", ".": "!"}, it would return "Jello world!".

    Keyword arguments:
    string       -- The string to replace characters in.
    mappings     -- A dictionary of replacement mappings.
    lower_keys   -- Whether or not to lower the keys in mappings.
    lower_values -- Whether or not to lower the values in mappings.
    lower_string -- Whether or not to lower the input string.
    """
    replaced_string = string.lower() if lower_string else string
    for character, replacement in mappings.items():
        replaced_string = replaced_string.replace(
            character.lower() if lower_keys else character,
            replacement.lower() if lower_values else replacement
        )
    return replaced_string

In [49]:
for key in word_dict:
    vowelless.append(keymap_replace(key, replace_dict))

In [51]:
#now we do the same thing as above where the frequencies of homophones are added together
homophones = []
nonhomophones = []
for w in vowelless:
    if vowelless.count(w) > 1:
        homophones.append(w)
    else:
        nonhomophones.append(w)


In [57]:
#put frequencies in a list
freq = list(word_dict.values())


In [62]:
#get unique homophones
homophones = set(homophones)
#get the indices for all the homophones
indexPosList = []
for h in homophones:
    indexPosList.append([ i for i in range(len(vowelless)) if vowelless[i] == h ])
#get the frequencies at those indices
hfreq = []
localfreq = []
for j in range(len(indexPosList)):
    for i in range(len(indexPosList[j])):
        localfreq.append(freq[indexPosList[j][i]])
    hfreq.append(np.array(localfreq))
    localfreq = []
#sum the frequencies for each homophone together
hfreqsum = []
for row in hfreq:
    hfreqsum.append(np.sum(row, 0))
#make a dictionary of homophones, summed frequencies 
hdict = dict(zip(homophones, hfreqsum))



In [66]:
#make a dictionary of all words that are not homophones with their frequencies
#get the indices of all the nonhomophones in the word list
indexPosList2 = []
for h in nonhomophones:
    indexPosList2.append([ i for i in range(len(vowelless)) if vowelless[i] == h] )

#get the frequencies at those indices
nhfreq = []
for row in indexPosList2:
    nhfreq.append(freq[row[0]])
#make a dictionary of nonhomophones and frequencies
nhdict = dict(zip(nonhomophones, nhfreq))
#dictionary of all words, where each unique phonemic representation appears once 
#with the frequency of all homophones summed for that representation
vowelless_dict = {**hdict, **nhdict}

In [69]:
#entropy of system at the word level (token freq)

#first get total frequency so we can get the probability
totalfreq = sum(vowelless_dict.values())

#now divide the frequency of each word by the total frequency to get the probability
probabilities = {}
for key in vowelless_dict:
    probabilities[key] = vowelless_dict[key]/totalfreq
    
#now get the surprisal for each vowel times the probability for each vowel
surprisals = {}
for key in probabilities:
    surprisals[key] = probabilities[key]*(math.log2(1/probabilities[key]))

#now add all of those together to get the entropy
entropy_vowelless_token = sum(surprisals.values())

In [70]:
entropy_vowelless_token

8.313873598607595

In [72]:
word_vowel_contr = (entropy_words_token - entropy_vowelless_token)/ entropy_words_token
word_vowel_contr

0.04336422517619309

## now some more fine-grained contrasts
## front vowels all merged and back vowels all merged


In [73]:

#First, replace all front vowels with 'i' and all back vowels with 'u'
replace_dict = {'I':'i', 'e':'i', 'E':'i', '@':'i', 'R':'i', 'U':'u', 'o':'u', 'c':'u', 'a':'u', 'V':'u'}
vowelless = []

In [74]:
# a function I found from https://codereview.stackexchange.com/questions/97318/string-replacement-using-dictionaries
def keymap_replace(
        string: str, 
        mappings: dict,
        lower_keys=False,
        lower_values=False,
        lower_string=False,
    ) -> str:
    """Replace parts of a string based on a dictionary.

    This function takes a string a dictionary of
    replacement mappings. For example, if I supplied
    the string "Hello world.", and the mappings 
    {"H": "J", ".": "!"}, it would return "Jello world!".

    Keyword arguments:
    string       -- The string to replace characters in.
    mappings     -- A dictionary of replacement mappings.
    lower_keys   -- Whether or not to lower the keys in mappings.
    lower_values -- Whether or not to lower the values in mappings.
    lower_string -- Whether or not to lower the input string.
    """
    replaced_string = string.lower() if lower_string else string
    for character, replacement in mappings.items():
        replaced_string = replaced_string.replace(
            character.lower() if lower_keys else character,
            replacement.lower() if lower_values else replacement
        )
    return replaced_string

In [75]:
for key in word_dict:
    vowelless.append(keymap_replace(key, replace_dict))

In [76]:
#now we do the same thing as above where the frequencies of homophones are added together
homophones = []
nonhomophones = []
for w in vowelless:
    if vowelless.count(w) > 1:
        homophones.append(w)
    else:
        nonhomophones.append(w)


In [77]:
#put frequencies in a list
freq = list(word_dict.values())


In [78]:
#get unique homophones
homophones = set(homophones)
#get the indices for all the homophones
indexPosList = []
for h in homophones:
    indexPosList.append([ i for i in range(len(vowelless)) if vowelless[i] == h ])
#get the frequencies at those indices
hfreq = []
localfreq = []
for j in range(len(indexPosList)):
    for i in range(len(indexPosList[j])):
        localfreq.append(freq[indexPosList[j][i]])
    hfreq.append(np.array(localfreq))
    localfreq = []
#sum the frequencies for each homophone together
hfreqsum = []
for row in hfreq:
    hfreqsum.append(np.sum(row, 0))
#make a dictionary of homophones, summed frequencies 
hdict = dict(zip(homophones, hfreqsum))



In [79]:
#make a dictionary of all words that are not homophones with their frequencies
#get the indices of all the nonhomophones in the word list
indexPosList2 = []
for h in nonhomophones:
    indexPosList2.append([ i for i in range(len(vowelless)) if vowelless[i] == h] )

#get the frequencies at those indices
nhfreq = []
for row in indexPosList2:
    nhfreq.append(freq[row[0]])
#make a dictionary of nonhomophones and frequencies
nhdict = dict(zip(nonhomophones, nhfreq))
#dictionary of all words, where each unique phonemic representation appears once 
#with the frequency of all homophones summed for that representation
vowelless_dict = {**hdict, **nhdict}

In [80]:
#entropy of system at the word level (token freq)

#first get total frequency so we can get the probability
totalfreq = sum(vowelless_dict.values())

#now divide the frequency of each word by the total frequency to get the probability
probabilities = {}
for key in vowelless_dict:
    probabilities[key] = vowelless_dict[key]/totalfreq
    
#now get the surprisal for each vowel times the probability for each vowel
surprisals = {}
for key in probabilities:
    surprisals[key] = probabilities[key]*(math.log2(1/probabilities[key]))

#now add all of those together to get the entropy
entropy_vowelless_token = sum(surprisals.values())

In [81]:
entropy_vowelless_token

8.466481589956453

In [82]:
word_vowel_contr = (entropy_words_token - entropy_vowelless_token)/ entropy_words_token
word_vowel_contr

0.025804388318344935

## and now high vs low vowels

In [83]:
#replace all high vowels with 'i' and all low vowels with 'E'
replace_dict = {'e': 'i', 'I': 'i', 'u': 'i', 'U': 'i', 'o': 'i', '@': 'E', 'a': 'E', 'c': 'E', 'R': 'E', 'V': 'E'}
vowelless = []

In [84]:
# a function I found from https://codereview.stackexchange.com/questions/97318/string-replacement-using-dictionaries
def keymap_replace(
        string: str, 
        mappings: dict,
        lower_keys=False,
        lower_values=False,
        lower_string=False,
    ) -> str:
    """Replace parts of a string based on a dictionary.

    This function takes a string a dictionary of
    replacement mappings. For example, if I supplied
    the string "Hello world.", and the mappings 
    {"H": "J", ".": "!"}, it would return "Jello world!".

    Keyword arguments:
    string       -- The string to replace characters in.
    mappings     -- A dictionary of replacement mappings.
    lower_keys   -- Whether or not to lower the keys in mappings.
    lower_values -- Whether or not to lower the values in mappings.
    lower_string -- Whether or not to lower the input string.
    """
    replaced_string = string.lower() if lower_string else string
    for character, replacement in mappings.items():
        replaced_string = replaced_string.replace(
            character.lower() if lower_keys else character,
            replacement.lower() if lower_values else replacement
        )
    return replaced_string

In [85]:
for key in word_dict:
    vowelless.append(keymap_replace(key, replace_dict))

In [86]:
#now we do the same thing as above where the frequencies of homophones are added together
homophones = []
nonhomophones = []
for w in vowelless:
    if vowelless.count(w) > 1:
        homophones.append(w)
    else:
        nonhomophones.append(w)


In [87]:
#put frequencies in a list
freq = list(word_dict.values())


In [88]:
#get unique homophones
homophones = set(homophones)
#get the indices for all the homophones
indexPosList = []
for h in homophones:
    indexPosList.append([ i for i in range(len(vowelless)) if vowelless[i] == h ])
#get the frequencies at those indices
hfreq = []
localfreq = []
for j in range(len(indexPosList)):
    for i in range(len(indexPosList[j])):
        localfreq.append(freq[indexPosList[j][i]])
    hfreq.append(np.array(localfreq))
    localfreq = []
#sum the frequencies for each homophone together
hfreqsum = []
for row in hfreq:
    hfreqsum.append(np.sum(row, 0))
#make a dictionary of homophones, summed frequencies 
hdict = dict(zip(homophones, hfreqsum))



In [89]:
#make a dictionary of all words that are not homophones with their frequencies
#get the indices of all the nonhomophones in the word list
indexPosList2 = []
for h in nonhomophones:
    indexPosList2.append([ i for i in range(len(vowelless)) if vowelless[i] == h] )

#get the frequencies at those indices
nhfreq = []
for row in indexPosList2:
    nhfreq.append(freq[row[0]])
#make a dictionary of nonhomophones and frequencies
nhdict = dict(zip(nonhomophones, nhfreq))
#dictionary of all words, where each unique phonemic representation appears once 
#with the frequency of all homophones summed for that representation
vowelless_dict = {**hdict, **nhdict}

In [90]:
#entropy of system at the word level (token freq)

#first get total frequency so we can get the probability
totalfreq = sum(vowelless_dict.values())

#now divide the frequency of each word by the total frequency to get the probability
probabilities = {}
for key in vowelless_dict:
    probabilities[key] = vowelless_dict[key]/totalfreq
    
#now get the surprisal for each vowel times the probability for each vowel
surprisals = {}
for key in probabilities:
    surprisals[key] = probabilities[key]*(math.log2(1/probabilities[key]))

#now add all of those together to get the entropy
entropy_vowelless_token = sum(surprisals.values())

In [91]:
entropy_vowelless_token

8.512109186458147

In [92]:
word_vowel_contr = (entropy_words_token - entropy_vowelless_token)/ entropy_words_token
word_vowel_contr

0.02055424942519935

## and tense vs lax

In [93]:
#replace all tense vowels with 'i' and all lax vowels with 'I'
replace_dict = {'e':'i', 'u':'i', 'a':'i', 'o':'i', 'c':'i', 'E':'I', 'U':'I', 'V':'I', '@':'I', 'R':'I' }
vowelless = []

In [94]:
# a function I found from https://codereview.stackexchange.com/questions/97318/string-replacement-using-dictionaries
def keymap_replace(
        string: str, 
        mappings: dict,
        lower_keys=False,
        lower_values=False,
        lower_string=False,
    ) -> str:
    """Replace parts of a string based on a dictionary.

    This function takes a string a dictionary of
    replacement mappings. For example, if I supplied
    the string "Hello world.", and the mappings 
    {"H": "J", ".": "!"}, it would return "Jello world!".

    Keyword arguments:
    string       -- The string to replace characters in.
    mappings     -- A dictionary of replacement mappings.
    lower_keys   -- Whether or not to lower the keys in mappings.
    lower_values -- Whether or not to lower the values in mappings.
    lower_string -- Whether or not to lower the input string.
    """
    replaced_string = string.lower() if lower_string else string
    for character, replacement in mappings.items():
        replaced_string = replaced_string.replace(
            character.lower() if lower_keys else character,
            replacement.lower() if lower_values else replacement
        )
    return replaced_string

In [95]:
for key in word_dict:
    vowelless.append(keymap_replace(key, replace_dict))

In [96]:
#now we do the same thing as above where the frequencies of homophones are added together
homophones = []
nonhomophones = []
for w in vowelless:
    if vowelless.count(w) > 1:
        homophones.append(w)
    else:
        nonhomophones.append(w)


In [97]:
#put frequencies in a list
freq = list(word_dict.values())


In [98]:
#get unique homophones
homophones = set(homophones)
#get the indices for all the homophones
indexPosList = []
for h in homophones:
    indexPosList.append([ i for i in range(len(vowelless)) if vowelless[i] == h ])
#get the frequencies at those indices
hfreq = []
localfreq = []
for j in range(len(indexPosList)):
    for i in range(len(indexPosList[j])):
        localfreq.append(freq[indexPosList[j][i]])
    hfreq.append(np.array(localfreq))
    localfreq = []
#sum the frequencies for each homophone together
hfreqsum = []
for row in hfreq:
    hfreqsum.append(np.sum(row, 0))
#make a dictionary of homophones, summed frequencies 
hdict = dict(zip(homophones, hfreqsum))



In [99]:
#make a dictionary of all words that are not homophones with their frequencies
#get the indices of all the nonhomophones in the word list
indexPosList2 = []
for h in nonhomophones:
    indexPosList2.append([ i for i in range(len(vowelless)) if vowelless[i] == h] )

#get the frequencies at those indices
nhfreq = []
for row in indexPosList2:
    nhfreq.append(freq[row[0]])
#make a dictionary of nonhomophones and frequencies
nhdict = dict(zip(nonhomophones, nhfreq))
#dictionary of all words, where each unique phonemic representation appears once 
#with the frequency of all homophones summed for that representation
vowelless_dict = {**hdict, **nhdict}

In [100]:
#entropy of system at the word level (token freq)

#first get total frequency so we can get the probability
totalfreq = sum(vowelless_dict.values())

#now divide the frequency of each word by the total frequency to get the probability
probabilities = {}
for key in vowelless_dict:
    probabilities[key] = vowelless_dict[key]/totalfreq
    
#now get the surprisal for each vowel times the probability for each vowel
surprisals = {}
for key in probabilities:
    surprisals[key] = probabilities[key]*(math.log2(1/probabilities[key]))

#now add all of those together to get the entropy
entropy_vowelless_token = sum(surprisals.values())

In [101]:
entropy_vowelless_token

8.481174634919173

In [102]:
word_vowel_contr = (entropy_words_token - entropy_vowelless_token)/ entropy_words_token
word_vowel_contr

0.024113733260179755

## a c merger

In [158]:
#replace 'c' with 'a'
replace_dict = {'c': 'a'}
vowelless = []

In [159]:
for key in word_dict:
    vowelless.append(keymap_replace(key, replace_dict))

In [160]:
#now we do the same thing as above where the frequencies of homophones are added together
homophones = []
nonhomophones = []
for w in vowelless:
    if vowelless.count(w) > 1:
        homophones.append(w)
    else:
        nonhomophones.append(w)


In [161]:
#put frequencies in a list
freq = list(word_dict.values())


In [162]:
#get unique homophones
homophones = set(homophones)
#get the indices for all the homophones
indexPosList = []
for h in homophones:
    indexPosList.append([ i for i in range(len(vowelless)) if vowelless[i] == h ])
#get the frequencies at those indices
hfreq = []
localfreq = []
for j in range(len(indexPosList)):
    for i in range(len(indexPosList[j])):
        localfreq.append(freq[indexPosList[j][i]])
    hfreq.append(np.array(localfreq))
    localfreq = []
#sum the frequencies for each homophone together
hfreqsum = []
for row in hfreq:
    hfreqsum.append(np.sum(row, 0))
#make a dictionary of homophones, summed frequencies 
hdict = dict(zip(homophones, hfreqsum))



In [163]:
#make a dictionary of all words that are not homophones with their frequencies
#get the indices of all the nonhomophones in the word list
indexPosList2 = []
for h in nonhomophones:
    indexPosList2.append([ i for i in range(len(vowelless)) if vowelless[i] == h] )

#get the frequencies at those indices
nhfreq = []
for row in indexPosList2:
    nhfreq.append(freq[row[0]])
#make a dictionary of nonhomophones and frequencies
nhdict = dict(zip(nonhomophones, nhfreq))
#dictionary of all words, where each unique phonemic representation appears once 
#with the frequency of all homophones summed for that representation
vowelless_dict = {**hdict, **nhdict}

In [164]:
#entropy of system at the word level (token freq)

#first get total frequency so we can get the probability
totalfreq = sum(vowelless_dict.values())

#now divide the frequency of each word by the total frequency to get the probability
probabilities = {}
for key in vowelless_dict:
    probabilities[key] = vowelless_dict[key]/totalfreq
    
#now get the surprisal for each vowel times the probability for each vowel
surprisals = {}
for key in probabilities:
    surprisals[key] = probabilities[key]*(math.log2(1/probabilities[key]))

#now add all of those together to get the entropy
entropy_vowelless_token = sum(surprisals.values())

In [165]:
entropy_vowelless_token

8.690078308858006

In [166]:
word_vowel_contr = (entropy_words_token - entropy_vowelless_token)/ entropy_words_token
word_vowel_contr

7.623429994954556e-05

## i I merger

In [167]:
#replace 'I' with 'i'
replace_dict = {'I': 'i'}
vowelless = []

In [168]:
for key in word_dict:
    vowelless.append(keymap_replace(key, replace_dict))

In [169]:
#now we do the same thing as above where the frequencies of homophones are added together
homophones = []
nonhomophones = []
for w in vowelless:
    if vowelless.count(w) > 1:
        homophones.append(w)
    else:
        nonhomophones.append(w)


In [170]:
#put frequencies in a list
freq = list(word_dict.values())


In [171]:
#get unique homophones
homophones = set(homophones)
#get the indices for all the homophones
indexPosList = []
for h in homophones:
    indexPosList.append([ i for i in range(len(vowelless)) if vowelless[i] == h ])
#get the frequencies at those indices
hfreq = []
localfreq = []
for j in range(len(indexPosList)):
    for i in range(len(indexPosList[j])):
        localfreq.append(freq[indexPosList[j][i]])
    hfreq.append(np.array(localfreq))
    localfreq = []
#sum the frequencies for each homophone together
hfreqsum = []
for row in hfreq:
    hfreqsum.append(np.sum(row, 0))
#make a dictionary of homophones, summed frequencies 
hdict = dict(zip(homophones, hfreqsum))



In [172]:
#make a dictionary of all words that are not homophones with their frequencies
#get the indices of all the nonhomophones in the word list
indexPosList2 = []
for h in nonhomophones:
    indexPosList2.append([ i for i in range(len(vowelless)) if vowelless[i] == h] )

#get the frequencies at those indices
nhfreq = []
for row in indexPosList2:
    nhfreq.append(freq[row[0]])
#make a dictionary of nonhomophones and frequencies
nhdict = dict(zip(nonhomophones, nhfreq))
#dictionary of all words, where each unique phonemic representation appears once 
#with the frequency of all homophones summed for that representation
vowelless_dict = {**hdict, **nhdict}

In [173]:
#entropy of system at the word level (token freq)

#first get total frequency so we can get the probability
totalfreq = sum(vowelless_dict.values())

#now divide the frequency of each word by the total frequency to get the probability
probabilities = {}
for key in vowelless_dict:
    probabilities[key] = vowelless_dict[key]/totalfreq
    
#now get the surprisal for each vowel times the probability for each vowel
surprisals = {}
for key in probabilities:
    surprisals[key] = probabilities[key]*(math.log2(1/probabilities[key]))

#now add all of those together to get the entropy
entropy_vowelless_token = sum(surprisals.values())

In [174]:
entropy_vowelless_token

8.683668374498042

In [175]:
word_vowel_contr = (entropy_words_token - entropy_vowelless_token)/ entropy_words_token
word_vowel_contr

0.0008137933270726189

## u U merger

In [176]:
#replace 'U' with 'u'
replace_dict = {'U': 'u'}
vowelless = []

In [177]:
for key in word_dict:
    vowelless.append(keymap_replace(key, replace_dict))

In [178]:
#now we do the same thing as above where the frequencies of homophones are added together
homophones = []
nonhomophones = []
for w in vowelless:
    if vowelless.count(w) > 1:
        homophones.append(w)
    else:
        nonhomophones.append(w)


In [179]:
#put frequencies in a list
freq = list(word_dict.values())


In [180]:
#get unique homophones
homophones = set(homophones)
#get the indices for all the homophones
indexPosList = []
for h in homophones:
    indexPosList.append([ i for i in range(len(vowelless)) if vowelless[i] == h ])
#get the frequencies at those indices
hfreq = []
localfreq = []
for j in range(len(indexPosList)):
    for i in range(len(indexPosList[j])):
        localfreq.append(freq[indexPosList[j][i]])
    hfreq.append(np.array(localfreq))
    localfreq = []
#sum the frequencies for each homophone together
hfreqsum = []
for row in hfreq:
    hfreqsum.append(np.sum(row, 0))
#make a dictionary of homophones, summed frequencies 
hdict = dict(zip(homophones, hfreqsum))



In [181]:
#make a dictionary of all words that are not homophones with their frequencies
#get the indices of all the nonhomophones in the word list
indexPosList2 = []
for h in nonhomophones:
    indexPosList2.append([ i for i in range(len(vowelless)) if vowelless[i] == h] )

#get the frequencies at those indices
nhfreq = []
for row in indexPosList2:
    nhfreq.append(freq[row[0]])
#make a dictionary of nonhomophones and frequencies
nhdict = dict(zip(nonhomophones, nhfreq))
#dictionary of all words, where each unique phonemic representation appears once 
#with the frequency of all homophones summed for that representation
vowelless_dict = {**hdict, **nhdict}

In [182]:
#entropy of system at the word level (token freq)

#first get total frequency so we can get the probability
totalfreq = sum(vowelless_dict.values())

#now divide the frequency of each word by the total frequency to get the probability
probabilities = {}
for key in vowelless_dict:
    probabilities[key] = vowelless_dict[key]/totalfreq
    
#now get the surprisal for each vowel times the probability for each vowel
surprisals = {}
for key in probabilities:
    surprisals[key] = probabilities[key]*(math.log2(1/probabilities[key]))

#now add all of those together to get the entropy
entropy_vowelless_token = sum(surprisals.values())

In [183]:
entropy_vowelless_token

8.69023087944811

In [184]:
word_vowel_contr = (entropy_words_token - entropy_vowelless_token)/ entropy_words_token
word_vowel_contr

5.867876666542823e-05

## U V merger

In [185]:
#replace 'V' with 'U'
replace_dict = {'V': 'U'}
vowelless = []

In [186]:
for key in word_dict:
    vowelless.append(keymap_replace(key, replace_dict))

In [187]:
#now we do the same thing as above where the frequencies of homophones are added together
homophones = []
nonhomophones = []
for w in vowelless:
    if vowelless.count(w) > 1:
        homophones.append(w)
    else:
        nonhomophones.append(w)


In [188]:
#put frequencies in a list
freq = list(word_dict.values())


In [189]:
#get unique homophones
homophones = set(homophones)
#get the indices for all the homophones
indexPosList = []
for h in homophones:
    indexPosList.append([ i for i in range(len(vowelless)) if vowelless[i] == h ])
#get the frequencies at those indices
hfreq = []
localfreq = []
for j in range(len(indexPosList)):
    for i in range(len(indexPosList[j])):
        localfreq.append(freq[indexPosList[j][i]])
    hfreq.append(np.array(localfreq))
    localfreq = []
#sum the frequencies for each homophone together
hfreqsum = []
for row in hfreq:
    hfreqsum.append(np.sum(row, 0))
#make a dictionary of homophones, summed frequencies 
hdict = dict(zip(homophones, hfreqsum))



In [190]:
#make a dictionary of all words that are not homophones with their frequencies
#get the indices of all the nonhomophones in the word list
indexPosList2 = []
for h in nonhomophones:
    indexPosList2.append([ i for i in range(len(vowelless)) if vowelless[i] == h] )

#get the frequencies at those indices
nhfreq = []
for row in indexPosList2:
    nhfreq.append(freq[row[0]])
#make a dictionary of nonhomophones and frequencies
nhdict = dict(zip(nonhomophones, nhfreq))
#dictionary of all words, where each unique phonemic representation appears once 
#with the frequency of all homophones summed for that representation
vowelless_dict = {**hdict, **nhdict}

In [191]:
#entropy of system at the word level (token freq)

#first get total frequency so we can get the probability
totalfreq = sum(vowelless_dict.values())

#now divide the frequency of each word by the total frequency to get the probability
probabilities = {}
for key in vowelless_dict:
    probabilities[key] = vowelless_dict[key]/totalfreq
    
#now get the surprisal for each vowel times the probability for each vowel
surprisals = {}
for key in probabilities:
    surprisals[key] = probabilities[key]*(math.log2(1/probabilities[key]))

#now add all of those together to get the entropy
entropy_vowelless_token = sum(surprisals.values())

In [192]:
entropy_vowelless_token

8.689595295704388

In [193]:
word_vowel_contr = (entropy_words_token - entropy_vowelless_token)/ entropy_words_token
word_vowel_contr

0.00013181220319534486